In [1]:
rm(list = ls())

library(readr)
library(dplyr)
library(car)
library(Hmisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


Loading required package: lattice

Loading required package: survival

Warning message:
“package ‘survival’ was built under R version 4.1.1”
Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units




## Question 1

### #1. load data

In [2]:
data <- read_csv("mba3.csv")
data <- na.omit(data)

Rows: 250 Columns: 5

── Column specification ──────────────────────────────────────────────────────────────
Delimiter: ","
dbl (5): salary, testsc, wam, libsize, rank


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



### #2. check the head of the table

In [3]:
head(data)

salary,testsc,wam,libsize,rank
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
131400,10.33,3.15,216,128
133098,10.67,3.50,256,104
132870,10.33,3.25,424,34
135000,10.47,3.20,329,49
133606,10.80,3.38,332,95
131700,10.73,3.40,311,98


### #3. find the summary statistics

In [4]:
summary(data)

     salary           testsc           wam           libsize      
 Min.   :124900   Min.   : 9.67   Min.   :2.730   Min.   : 124.0  
 1st Qu.:130150   1st Qu.:10.33   1st Qu.:3.200   1st Qu.: 230.2  
 Median :134400   Median :10.53   Median :3.300   Median : 310.5  
 Mean   :139752   Mean   :10.59   Mean   :3.314   Mean   : 354.9  
 3rd Qu.:142300   3rd Qu.:10.80   3rd Qu.:3.420   3rd Qu.: 409.8  
 Max.   :179000   Max.   :11.40   Max.   :3.820   Max.   :1745.0  
      rank       
 Min.   :  1.00  
 1st Qu.: 38.00  
 Median : 79.00  
 Mean   : 79.54  
 3rd Qu.:122.75  
 Max.   :174.00  

### if you want to wokr it one by one

In [5]:
min(data$salary  )
min(data$testsc  )
min(data$wam     )
min(data$libsize )
min(data$rank    )

[1] 124900

[1] 9.67

[1] 2.73

[1] 124

[1] 1

In [6]:
max(data$salary  )
max(data$testsc  )
max(data$wam     )
max(data$libsize )
max(data$rank    )

[1] 179000

[1] 11.4

[1] 3.82

[1] 1745

[1] 174

In [7]:
mean(data$salary  )
mean(data$testsc  )
mean(data$wam     )
mean(data$libsize )
mean(data$rank    )

[1] 139752

[1] 10.58688

[1] 3.31416

[1] 354.928

[1] 79.54

In [8]:
sd(data$salary  )
sd(data$testsc  )
sd(data$wam     )
sd(data$libsize )
sd(data$rank    )

[1] 12976.66

[1] 0.294749

[1] 0.1979278

[1] 211.8442

[1] 48.4889

### #4. OLS of log(salary)

In [9]:
data$log_salary  = log(data$salary  )
data$log_libsize = log(data$libsize )

OLS1 = lm(log_salary ~  testsc + wam + log_libsize + rank , data=data)

summary(OLS1)


Call:
lm(formula = log_salary ~ testsc + wam + log_libsize + rank, 
    data = data)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.091265 -0.033113 -0.002759  0.028784  0.102116 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.100e+01  1.431e-01  76.877  < 2e-16 ***
testsc       5.011e-02  1.670e-02   3.001  0.00297 ** 
wam          4.803e-02  2.338e-02   2.054  0.04099 *  
log_libsize  3.969e-02  8.861e-03   4.479 1.15e-05 ***
rank        -9.365e-04  9.003e-05 -10.402  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.04044 on 245 degrees of freedom
Multiple R-squared:  0.7927,	Adjusted R-squared:  0.7894 
F-statistic: 234.3 on 4 and 245 DF,  p-value: < 2.2e-16


### #5. t-test on $$H_0 : \beta_{\mathrm{rank}}=0;$$ $$H_1 : \beta_{\mathrm{rank}} \neq 0;$$

### note that
### * when testing single parameter (like $\beta_{rank}$) in *a two tail test*, you can do either F or t test
### * the *two-tail p-value* for F and t are the same
### * the F and t values always follows this rule $$F = t^2$$
### * hence, if you know F, you also know the corresponding t in *two-tail tests*.

In [10]:
linearHypothesis(OLS1, c("testsc = 0"),test = "F")

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,246,0.4153109,NA,NA,NA,NA
2,245,0.4005823,1,0.01472856,9.00813,0.00296563


### t-test on $$H_0 : \beta_{\mathrm{rank}} \geqslant 0;$$ $$H_1 : \beta_{\mathrm{rank}} < 0;$$

In [11]:
hyp_val = 0
t_val = (summary(OLS1)$coefficients[5,1] - hyp_val ) / summary(OLS1)$coefficients[5,2]

p_val = pt(t_val, df.residual(OLS1))

print(paste0("p value is ", p_val))

[1] "p value is 1.58259773181506e-21"


### #6. F test on $$H_0 : \beta_{\mathrm{wam}}=\beta_{\mathrm{testsc}}=0;$$ $$H_1 : \mbox{ at least one of } \beta_{\mathrm{wam}} \mbox{ and }\beta_{\mathrm{testsc}} \mbox{ is not } 0.$$ 

In [12]:
linearHypothesis(OLS1, c("testsc= 0","wam = 0"),test = "F")

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,247,0.4551063,NA,NA,NA,NA
2,245,0.4005823,2,0.05452398,16.67369,1.625248e-07


### #7. F test on $$H_0 : \beta_{\mathrm{wam}}=\beta_{\mathrm{testsc}};$$ $$H_1 : \beta_{\mathrm{wam}} \neq \beta_{\mathrm{testsc}}.$$

In [13]:
linearHypothesis(OLS1, c("testsc - wam = 0"),test = "F")

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,246,0.4005878,NA,NA,NA,NA
2,245,0.4005823,1,5.442344e-06,0.00332859,0.9540395


## Extra : do all the things again on the standardized regression and you will find the same result

In [14]:
data_st = select(data, log_salary, testsc, wam, log_libsize, rank)

data_st$log_salary  = (data_st$log_salary  - mean(data_st$log_salary ))/sd(data_st$log_salary )
data_st$testsc      = (data_st$testsc      - mean(data_st$testsc     ))/sd(data_st$testsc)
data_st$wam         = (data_st$wam         - mean(data_st$wam        ))/sd(data_st$wam)
data_st$log_libsize = (data_st$log_libsize - mean(data_st$log_libsize))/sd(data_st$log_libsize)
data_st$rank        = (data_st$rank        - mean(data_st$rank       ))/sd(data_st$rank)

OLS2 = lm(log_salary ~  testsc + wam + log_libsize + rank, data=data_st)
summary(OLS2)


Call:
lm(formula = log_salary ~ testsc + wam + log_libsize + rank, 
    data = data_st)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.03589 -0.37584 -0.03132  0.32671  1.15905 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -6.643e-15  2.903e-02   0.000  1.00000    
testsc       1.676e-01  5.586e-02   3.001  0.00297 ** 
wam          1.079e-01  5.252e-02   2.054  0.04099 *  
log_libsize  1.965e-01  4.387e-02   4.479 1.15e-05 ***
rank        -5.154e-01  4.955e-02 -10.402  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.459 on 245 degrees of freedom
Multiple R-squared:  0.7927,	Adjusted R-squared:  0.7894 
F-statistic: 234.3 on 4 and 245 DF,  p-value: < 2.2e-16


In [15]:
linearHypothesis(OLS2, c("testsc = 0"),test = "F")

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,246,53.50447,NA,NA,NA,NA
2,245,51.60699,1,1.897479,9.00813,0.00296563


In [16]:
linearHypothesis(OLS1, c("testsc= 0","wam = 0"),test = "F")

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,247,0.4551063,NA,NA,NA,NA
2,245,0.4005823,2,0.05452398,16.67369,1.625248e-07


In [17]:
linearHypothesis(OLS1, c("testsc-wam = 0"),test = "F")

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,246,0.4005878,NA,NA,NA,NA
2,245,0.4005823,1,5.442344e-06,0.00332859,0.9540395


## Question 2

### #1. load data

In [18]:
data <- read_csv("nbasal.csv")
data <- na.omit(data)

Rows: 269 Columns: 22

── Column specification ──────────────────────────────────────────────────────────────
Delimiter: ","
dbl (22): marr, wage, exper, age, coll, games, minutes, guard, forward, cent...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



### #2. list all variable names

In [19]:
colnames(data)

[1] "marr"     "wage"     "exper"    "age"      "coll"     "games"   
 [7] "minutes"  "guard"    "forward"  "center"   "points"   "rebounds"
[13] "assists"  "draft"    "allstar"  "avgmin"   "lwage"    "black"   
[19] "children" "expersq"  "agesq"    "marrblck"

### #3. check the table head

In [20]:
head(data)

marr,wage,exper,age,coll,games,minutes,guard,forward,center,⋯,assists,draft,allstar,avgmin,lwage,black,children,expersq,agesq,marrblck
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1002.5,4,27,4,77,2867,1,0,0,⋯,4.5,19,0,37.23376,6.910252,1,0,16,729,1
1,2030.0,5,28,4,78,2789,1,0,0,⋯,8.8,28,0,35.75641,7.615791,1,1,25,784,1
0,650.0,1,25,4,74,1149,0,0,1,⋯,0.2,19,0,15.52703,6.476973,1,0,1,625,0
0,2030.0,5,28,4,47,1178,0,1,0,⋯,1.5,1,0,25.06383,7.615791,1,0,25,784,0
0,755.0,3,24,4,82,2096,1,0,0,⋯,2.6,24,0,25.56098,6.626718,1,0,9,576,0
0,2014.5,9,31,4,82,1971,0,1,0,⋯,1.5,4,0,24.03658,7.608126,1,0,81,961,0


### #4. summary statistics

In [21]:
summary(data)

      marr             wage            exper             age       
 Min.   :0.0000   Min.   : 150.0   Min.   : 1.000   Min.   :21.00  
 1st Qu.:0.0000   1st Qu.: 783.5   1st Qu.: 2.000   1st Qu.:24.00  
 Median :0.0000   Median :1265.5   Median : 5.000   Median :27.00  
 Mean   :0.4458   Mean   :1532.7   Mean   : 5.304   Mean   :27.49  
 3rd Qu.:1.0000   3rd Qu.:2114.0   3rd Qu.: 8.000   3rd Qu.:30.00  
 Max.   :1.0000   Max.   :5740.0   Max.   :18.000   Max.   :41.00  
      coll           games          minutes         guard       
 Min.   :0.000   Min.   : 6.00   Min.   :  66   Min.   :0.0000  
 1st Qu.:4.000   1st Qu.:63.00   1st Qu.:1145   1st Qu.:0.0000  
 Median :4.000   Median :75.00   Median :1859   Median :0.0000  
 Mean   :3.721   Mean   :67.73   Mean   :1788   Mean   :0.4167  
 3rd Qu.:4.000   3rd Qu.:80.00   3rd Qu.:2544   3rd Qu.:1.0000  
 Max.   :4.000   Max.   :82.00   Max.   :3533   Max.   :1.0000  
    forward           center           points         rebounds     
 

### #5. regression of points without $age^2$

In [22]:
OLS1 = lm(points ~  exper + expersq + age + coll , data=data)

summary(OLS1)


Call:
lm(formula = points ~ exper + expersq + age + coll, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-11.6278  -3.9174  -0.7898   3.4843  20.4819 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 34.85105    7.35154   4.741 3.69e-06 ***
exper        2.18075    0.43708   4.989 1.18e-06 ***
expersq     -0.07275    0.02439  -2.983  0.00316 ** 
age         -0.99407    0.31796  -3.126  0.00199 ** 
coll        -1.44748    0.50272  -2.879  0.00435 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.507 on 235 degrees of freedom
Multiple R-squared:  0.1263,	Adjusted R-squared:  0.1115 
F-statistic: 8.496 on 4 and 235 DF,  p-value: 2.029e-06


### #6. regression of points with $age^2$

In [23]:
OLS2 = lm(points ~  exper + expersq + age + agesq + coll , data=data)

summary(OLS2)


Call:
lm(formula = points ~ exper + expersq + age + agesq + coll, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-11.5188  -3.9265  -0.7557   3.3711  20.4080 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 56.22823   37.83898   1.486 0.138629    
exper        2.47291    0.67000   3.691 0.000278 ***
expersq     -0.10158    0.05570  -1.824 0.069461 .  
age         -2.61873    2.83872  -0.923 0.357215    
agesq        0.02982    0.05178   0.576 0.565197    
coll        -1.44952    0.50345  -2.879 0.004356 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.515 on 234 degrees of freedom
Multiple R-squared:  0.1276,	Adjusted R-squared:  0.1089 
F-statistic: 6.844 on 5 and 234 DF,  p-value: 5.575e-06


### #7. regression of lwage

In [24]:
OLS3 = lm(lwage ~  points + exper + expersq + age + coll , data=data)

summary(OLS3)


Call:
lm(formula = lwage ~ points + exper + expersq + age + coll, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.00704 -0.23513  0.09779  0.38402  1.59842 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  6.423423   0.852263   7.537 1.05e-12 ***
points       0.071880   0.007225   9.949  < 2e-16 ***
exper        0.158941   0.050908   3.122  0.00202 ** 
expersq     -0.005783   0.002752  -2.101  0.03667 *  
age         -0.016436   0.035941  -0.457  0.64787    
coll        -0.076992   0.056652  -1.359  0.17544    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6099 on 234 degrees of freedom
Multiple R-squared:  0.4443,	Adjusted R-squared:  0.4325 
F-statistic: 37.42 on 5 and 234 DF,  p-value: < 2.2e-16


### #8. the silly kitchen sink regression (center is dropped automatically due to perfect collinearity)

In [25]:
OLS4 = lm(lwage ~ marr + exper + age + coll + games + minutes + guard + forward + center + points + rebounds + assists + draft + allstar + avgmin + black + children + expersq + agesq + marrblck, data=data)

summary(OLS4)


Call:
lm(formula = lwage ~ marr + exper + age + coll + games + minutes + 
    guard + forward + center + points + rebounds + assists + 
    draft + allstar + avgmin + black + children + expersq + agesq + 
    marrblck, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.1698 -0.2461  0.1012  0.3118  1.6591 

Coefficients: (1 not defined because of singularities)
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 11.3715881  4.0120155   2.834  0.00502 ** 
marr         0.0962407  0.1778301   0.541  0.58892    
exper        0.2067546  0.0728500   2.838  0.00496 ** 
age         -0.4476439  0.2985100  -1.500  0.13515    
coll        -0.0627516  0.0539918  -1.162  0.24640    
games        0.0039273  0.0051026   0.770  0.44232    
minutes     -0.0002913  0.0002319  -1.256  0.21037    
guard       -0.1778261  0.1497650  -1.187  0.23636    
forward     -0.0238521  0.1115409  -0.214  0.83087    
center              NA         NA      NA       NA    
points      